In [ ]:
from google.colab import files

uploaded = files.upload()

Saving abcnews-date-text.csv to abcnews-date-text.csv


In [ ]:
import pandas as pd
data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False);
print(data)

<ipython-input-6-bd024a9bc065>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False);


         publish_date                                      headline_text
0            20030219  aba decides against community broadcasting lic...
1            20030219     act fire witnesses must be aware of defamation
2            20030219     a g calls for infrastructure protection summit
3            20030219           air nz staff in aust strike for pay rise
4            20030219      air nz strike to affect australian travellers
...               ...                                                ...
1244179      20211231  two aged care residents die as state records 2...
1244180      20211231  victoria records 5;919 new cases and seven deaths
1244181      20211231    wa delays adopting new close contact definition
1244182      20211231  western ringtail possums found badly dehydrate...
1244183      20211231  what makes you a close covid contact here are ...

[1244184 rows x 2 columns]


In [ ]:
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text
print(documents)

                                             headline_text    index
0        aba decides against community broadcasting lic...        0
1           act fire witnesses must be aware of defamation        1
2           a g calls for infrastructure protection summit        2
3                 air nz staff in aust strike for pay rise        3
4            air nz strike to affect australian travellers        4
...                                                    ...      ...
1244179  two aged care residents die as state records 2...  1244179
1244180  victoria records 5;919 new cases and seven deaths  1244180
1244181    wa delays adopting new close contact definition  1244181
1244182  western ringtail possums found badly dehydrate...  1244182
1244183  what makes you a close covid contact here are ...  1244183

[1244184 rows x 2 columns]


In [ ]:
print(len(documents))
print(documents[:5])

1244184
                                       headline_text  index
0  aba decides against community broadcasting lic...      0
1     act fire witnesses must be aware of defamation      1
2     a g calls for infrastructure protection summit      2
3           air nz staff in aust strike for pay rise      3
4      air nz strike to affect australian travellers      4


In [ ]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
stemmer = SnowballStemmer('english')
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [ ]:
doc_sample = documents[documents['index'] == 4310].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['ratepayers', 'group', 'wants', 'compulsory', 'local', 'govt', 'voting']


 tokenized and lemmatized document: 
['ratepay', 'group', 'want', 'compulsori', 'local', 'govt', 'vote']


In [ ]:
processed_docs = documents['headline_text'].map(preprocess)
processed_docs[:10]

0            [decid, communiti, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

In [ ]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 communiti
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


In [ ]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [ ]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(162, 1), (240, 1), (292, 1), (589, 1), (838, 1), (3571, 1), (3572, 1)]

In [ ]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5844216176085719),
 (1, 0.38716866963787633),
 (2, 0.5013820927104505),
 (3, 0.5071171375845095)]


In [ ]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.029*"vaccin" + 0.018*"health" + 0.017*"hospit" + 0.016*"scott" + 0.015*"farmer" + 0.012*"break" + 0.011*"show" + 0.010*"beach" + 0.010*"citi" + 0.010*"risk"
Topic: 1 Word: 0.043*"polic" + 0.029*"death" + 0.019*"crash" + 0.019*"home" + 0.018*"die" + 0.016*"lockdown" + 0.015*"perth" + 0.015*"shoot" + 0.014*"woman" + 0.014*"investig"
Topic: 2 Word: 0.027*"record" + 0.024*"test" + 0.023*"open" + 0.021*"australian" + 0.019*"market" + 0.015*"final" + 0.013*"year" + 0.011*"quarantin" + 0.010*"guilti" + 0.009*"india"
Topic: 3 Word: 0.031*"elect" + 0.018*"say" + 0.014*"state" + 0.014*"minist" + 0.013*"announc" + 0.013*"speak" + 0.012*"labor" + 0.011*"north" + 0.011*"call" + 0.011*"feder"
Topic: 4 Word: 0.046*"trump" + 0.045*"queensland" + 0.025*"donald" + 0.018*"canberra" + 0.018*"world" + 0.015*"time" + 0.013*"win" + 0.013*"premier" + 0.011*"assault" + 0.009*"sexual"
Topic: 5 Word: 0.039*"case" + 0.028*"charg" + 0.028*"court" + 0.023*"news" + 0.023*"murder" + 0.020*"face" + 0.

In [ ]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.010*"australia" + 0.010*"world" + 0.010*"final" + 0.008*"live" + 0.007*"leagu" + 0.007*"updat" + 0.006*"open" + 0.006*"cricket" + 0.006*"beat" + 0.005*"test"
Topic: 1 Word: 0.027*"trump" + 0.015*"countri" + 0.011*"hour" + 0.009*"border" + 0.009*"andrew" + 0.007*"christma" + 0.007*"coronavirus" + 0.007*"protest" + 0.007*"kill" + 0.005*"china"
Topic: 2 Word: 0.010*"health" + 0.009*"vaccin" + 0.009*"elect" + 0.008*"chang" + 0.008*"lockdown" + 0.008*"covid" + 0.007*"coronavirus" + 0.007*"say" + 0.007*"climat" + 0.006*"labor"
Topic: 3 Word: 0.011*"market" + 0.010*"morrison" + 0.009*"coronavirus" + 0.008*"royal" + 0.008*"price" + 0.008*"share" + 0.007*"rise" + 0.007*"australian" + 0.007*"bank" + 0.007*"commiss"
Topic: 4 Word: 0.015*"coronavirus" + 0.014*"queensland" + 0.013*"covid" + 0.012*"restrict" + 0.008*"stori" + 0.008*"drought" + 0.007*"australia" + 0.007*"farmer" + 0.007*"victoria" + 0.006*"juli"
Topic: 5 Word: 0.013*"bushfir" + 0.009*"search" + 0.008*"hous" + 0.008*"

In [ ]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.500542938709259	 
Topic: 0.034*"govern" + 0.023*"live" + 0.015*"worker" + 0.015*"indigen" + 0.014*"communiti" + 0.014*"work" + 0.012*"care" + 0.011*"street" + 0.011*"age" + 0.011*"support"

Score: 0.27015963196754456	 
Topic: 0.031*"elect" + 0.018*"say" + 0.014*"state" + 0.014*"minist" + 0.013*"announc" + 0.013*"speak" + 0.012*"labor" + 0.011*"north" + 0.011*"call" + 0.011*"feder"

Score: 0.14175541698932648	 
Topic: 0.027*"record" + 0.024*"test" + 0.023*"open" + 0.021*"australian" + 0.019*"market" + 0.015*"final" + 0.013*"year" + 0.011*"quarantin" + 0.010*"guilti" + 0.009*"india"

Score: 0.012507418170571327	 
Topic: 0.021*"nation" + 0.018*"adelaid" + 0.017*"tasmania" + 0.015*"victorian" + 0.013*"hous" + 0.013*"region" + 0.013*"school" + 0.013*"concern" + 0.013*"coast" + 0.012*"gold"

Score: 0.012505903840065002	 
Topic: 0.074*"coronavirus" + 0.069*"australia" + 0.065*"covid" + 0.031*"victoria" + 0.021*"south" + 0.017*"chang" + 0.015*"restrict" + 0.013*"island" + 0.012*"wate

In [ ]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.5062361359596252	 
Topic: 0.010*"health" + 0.009*"vaccin" + 0.009*"elect" + 0.008*"chang" + 0.008*"lockdown" + 0.008*"covid" + 0.007*"coronavirus" + 0.007*"say" + 0.007*"climat" + 0.006*"labor"

Score: 0.3937304615974426	 
Topic: 0.011*"market" + 0.010*"morrison" + 0.009*"coronavirus" + 0.008*"royal" + 0.008*"price" + 0.008*"share" + 0.007*"rise" + 0.007*"australian" + 0.007*"bank" + 0.007*"commiss"

Score: 0.012504803016781807	 
Topic: 0.021*"news" + 0.016*"rural" + 0.013*"drum" + 0.011*"govern" + 0.009*"friday" + 0.009*"monday" + 0.009*"age" + 0.008*"tuesday" + 0.008*"care" + 0.008*"wednesday"

Score: 0.012504634447395802	 
Topic: 0.015*"coronavirus" + 0.014*"queensland" + 0.013*"covid" + 0.012*"restrict" + 0.008*"stori" + 0.008*"drought" + 0.007*"australia" + 0.007*"farmer" + 0.007*"victoria" + 0.006*"juli"

Score: 0.012504566460847855	 
Topic: 0.027*"trump" + 0.015*"countri" + 0.011*"hour" + 0.009*"border" + 0.009*"andrew" + 0.007*"christma" + 0.007*"coronavirus" + 0.007*

In [ ]:
unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.3500807285308838	 Topic: 0.027*"record" + 0.024*"test" + 0.023*"open" + 0.021*"australian" + 0.019*"market"
Score: 0.3499029874801636	 Topic: 0.074*"coronavirus" + 0.069*"australia" + 0.065*"covid" + 0.031*"victoria" + 0.021*"south"
Score: 0.1832817941904068	 Topic: 0.032*"melbourn" + 0.023*"protest" + 0.022*"attack" + 0.016*"arrest" + 0.015*"kill"
Score: 0.016683083027601242	 Topic: 0.034*"govern" + 0.023*"live" + 0.015*"worker" + 0.015*"indigen" + 0.014*"communiti"
Score: 0.0166788250207901	 Topic: 0.031*"elect" + 0.018*"say" + 0.014*"state" + 0.014*"minist" + 0.013*"announc"
Score: 0.01667592115700245	 Topic: 0.039*"case" + 0.028*"charg" + 0.028*"court" + 0.023*"news" + 0.023*"murder"
Score: 0.016674157232046127	 Topic: 0.029*"vaccin" + 0.018*"health" + 0.017*"hospit" + 0.016*"scott" + 0.015*"farmer"
Score: 0.016674157232046127	 Topic: 0.043*"polic" + 0.029*"death" + 0.019*"crash" + 0.019*"home" + 0.018*"die"
Score: 0.016674157232046127	 Topic: 0.046*"trump" + 0.045*"queens